In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import resnet50
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.tensorboard import SummaryWriter

import pickle

In [1]:
# x_train
!gdown 1NurvaaNEbMDUO5O5hlHgs2VfS0vJcaZA

# y_train
!gdown 13xfi_KxHKZxlOgVSZbXwxTNajQ80XRyE

# x_test
!gdown 1w8pKD9JmQbkSXFCgqrumDxduFRe9Jpfr

# y_test
!gdown 166fdUzJJoNlOuIeIRZKML6sF4Tp4hh_t

Downloading...
From (original): https://drive.google.com/uc?id=1NurvaaNEbMDUO5O5hlHgs2VfS0vJcaZA
From (redirected): https://drive.google.com/uc?id=1NurvaaNEbMDUO5O5hlHgs2VfS0vJcaZA&confirm=t&uuid=ecdea420-cb6f-4aaa-ba49-8e4b498770a8
To: /content/x_train.pickle
100% 123M/123M [00:02<00:00, 48.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=13xfi_KxHKZxlOgVSZbXwxTNajQ80XRyE
To: /content/y_train.pickle
100% 26.2k/26.2k [00:00<00:00, 40.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w8pKD9JmQbkSXFCgqrumDxduFRe9Jpfr
To: /content/x_test.pickle
100% 40.9M/40.9M [00:00<00:00, 68.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=166fdUzJJoNlOuIeIRZKML6sF4Tp4hh_t
To: /content/y_test.pickle
100% 8.84k/8.84k [00:00<00:00, 21.5MB/s]


In [3]:
# x-train은 정규화, 리사이징, 증강 등 복잡한 전처리 저장
# y-train은 정수인코딩, 원핫인코딩 등 간단한 라벨처리
with open('x_train.pickle', 'rb') as f: #바이너리모드로 읽어서 파일객체에 할당
    x_train = pickle.load(f) # 바이너리 파일을 파이썬 객체에 복원

with open('y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)

with open('x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)

with open('y_test.pickle', 'rb') as f:
    y_test = pickle.load(f)

In [4]:
x_train.shape, y_train.shape

((3257, 112, 112, 3), (3257,))

In [5]:
x_test.shape, y_test.shape

((1086, 112, 112, 3), (1086,))

In [6]:
writer = SummaryWriter('runs/experiment') #훈련과정 중 수집한 로그를 기록 & 경로

In [7]:
# 데이터 전처리
def preprocess_data(images, labels):
    # int8 -> float32로 변환 및 정규화 (0-255 -> 0-1) (학습을 빠르고 안정적으로 하도록)
    images = torch.FloatTensor(images) / 255.0
    images = images.permute(0, 3, 1, 2) # 차원 순서를 지정 B-0 H-1 W-2 C-3 을 B C H W 로 바꿔줌
    labels = torch.LongTensor(labels) # LongTensor는 인티져형 64비트, 실수형 64비트는 double
    # PyTorch의 CrossEntropyLoss 함수는 정수형 클래스 인덱스를 요구함
    return images, labels

In [8]:
def get_resnet50(num_classes):
    model = resnet50(pretrained=True) # pretrained=True → ImageNet 데이터로 학습된 가중치를 사용함.
    model.fc = nn.Linear(model.fc.in_features, num_classes) # 말단 head에 유니크 중복제거 소환
    return model

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        # 학습 모드
        model.train() # 드롭아웃이나 배치 정규화 같은 레이어들이 학습 모드로 작동하게 설정.
        running_loss = 0.0
        correct = 0
        total = 0

        # 배치 단위로 학습
        for inputs, labels in train_loader:
            # 데이터를 device로 이동
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass (이미지를 모델에 이동시켜 예측값을 얻음)
            outputs = model(inputs)

            loss = criterion(outputs, labels) # 예측결과와 오차간 손실계산

            # Backward pass and optimize
            optimizer.zero_grad() # 기울기 초기화
            loss.backward() # 기울기계산(역전파)
            optimizer.step() # 계산된 기울기로 가중치 업데이트

            # 통계 업데이트
            running_loss += loss.item() #에폭 전체 누적 손실
            _, predicted = torch.max(outputs.data, 1) # 예측한 클래스의 인덱스
            total += labels.size(0) # 전체 샘플 누적갯수
            correct += (predicted == labels).sum().item() # 맞춘 누적갯수

        # 에폭별 학습 결과 출력
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

        # 검증
        model.eval() # 모델평가단계, 변수초기화
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%\n')

        # TensorBoard 기록
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/train', epoch_acc, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)

In [10]:
# 데이터 준비
train_images, train_labels = preprocess_data(x_train[:5000], y_train[:5000])
train_dataset = TensorDataset(train_images, train_labels) # 이미지와 레이블이 한 덩어리로 묶임

In [11]:
train_images.shape, train_labels.shape

(torch.Size([3257, 3, 112, 112]), torch.Size([3257]))

In [12]:
# 데이터 분할 (90% 학습, 10% 검증)
train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size]) #

In [13]:
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [14]:
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 선택 및 학습 (SimpleCNN)
num_classes = len(torch.unique(train_labels)) # unique는 중복제거
criterion = nn.CrossEntropyLoss()

In [ ]:
# ResNet50 학습
print("\nTraining ResNet50...")
resnet = get_resnet50(num_classes).to(device)
optimizer = optim.Adam(resnet.parameters(), lr=0.005)  # 더 작은 학습률 사용

train_model(resnet, train_loader, val_loader, criterion, optimizer, num_epochs=30, device=device)

writer.close()# 텐서보드 설정


Training ResNet50...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/30]
Training Loss: 1.1511, Accuracy: 58.42%
Validation Loss: 0.8218, Accuracy: 64.83%

Epoch [2/30]
Training Loss: 0.5030, Accuracy: 76.84%
Validation Loss: 0.8274, Accuracy: 65.64%

Epoch [3/30]
Training Loss: 0.4096, Accuracy: 81.21%
Validation Loss: 1.7966, Accuracy: 44.79%

Epoch [4/30]
Training Loss: 0.3457, Accuracy: 84.36%
Validation Loss: 1.7894, Accuracy: 52.56%

Epoch [5/30]
Training Loss: 0.3147, Accuracy: 86.16%
Validation Loss: 0.2306, Accuracy: 88.75%

Epoch [6/30]
Training Loss: 0.2993, Accuracy: 86.89%
Validation Loss: 0.2620, Accuracy: 87.53%

Epoch [7/30]
Training Loss: 0.2892, Accuracy: 87.93%
Validation Loss: 0.3630, Accuracy: 85.07%

Epoch [8/30]
Training Loss: 0.2452, Accuracy: 89.23%
Validation Loss: 0.3402, Accuracy: 85.28%

Epoch [9/30]
Training Loss: 0.2900, Accuracy: 88.44%
Validation Loss: 0.2333, Accuracy: 89.57%

Epoch [10/30]
Training Loss: 0.2214, Accuracy: 90.68%
Validation Loss: 1.7556, Accuracy: 69.53%

Epoch [11/30]
Training Loss: 0.2241, Ac

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs